In [1]:
# Define your data directories
train_dir = 'COP_banknote/ds3/Train'
test_dir = 'COP_banknote/ds3/Test'
validation_dir = 'COP_banknote/ds3/Validation'

In [2]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

2023-10-26 13:14:26.744483: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-26 13:14:26.764903: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-26 13:14:26.764918: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-26 13:14:26.764940: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-26 13:14:26.769711: I tensorflow/core/platform/cpu_feature_g

In [3]:
# Define image dimensions and batch size
image_size = (224, 224)
batch_size = 32

# Data augmentation for the training dataset
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load pre-trained MobileNetV2 model without top classification layer
base_model = MobileNetV2(
    input_shape=(image_size[0], image_size[1], 3),
    include_top=False,
    weights='imagenet'
)

# Add your custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)  # Add more dense layers if needed
predictions = Dense(12, activation='softmax')(x)  # Use 'softmax' for multi-class classification

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze all layers in the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model with 'categorical_crossentropy' for multi-class classification
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Create data generators for training, validation, and testing
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',  # Use 'categorical' for multi-class classification
    classes=['100k', '100kf', '10k', '10kf', '50k', '50kf', '5k', '5kf', '20k', '20kf', '2k', '2kf']
)

validation_datagen = ImageDataGenerator(rescale=1.0/255)
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',  # Use 'categorical' for multi-class classification
    classes=['100k', '100kf', '10k', '10kf', '50k', '50kf', '5k', '5kf', '20k', '20kf', '2k', '2kf']
)

test_datagen = ImageDataGenerator(rescale=1.0/255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',  # Use 'categorical' for multi-class classification
    classes=['100k', '100kf', '10k', '10kf', '50k', '50kf', '5k', '5kf', '20k', '20kf', '2k', '2kf']
)

# Train the model
num_epochs = 3
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    epochs=num_epochs
)

# Fine-tune the model by unfreezing some layers and training for more epochs if needed

# Evaluate the model on the test dataset
test_loss, test_acc = model.evaluate(test_generator, steps=len(test_generator))
print(f"Mobilenetv2_Test accuracy: {test_acc * 100:.2f}%")



2023-10-26 13:14:27.766148: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-26 13:14:27.783703: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-26 13:14:27.783838: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Found 8607 images belonging to 12 classes.
Found 2913 images belonging to 12 classes.
Found 2911 images belonging to 12 classes.
Epoch 1/3


2023-10-26 13:14:29.998924: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700
2023-10-26 13:14:30.074488: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-10-26 13:14:30.166893: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


  1/269 [..............................] - ETA: 6:34 - loss: 2.6658 - accuracy: 0.0938

2023-10-26 13:14:30.199643: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55c7352304d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-10-26 13:14:30.199662: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2023-10-26 13:14:30.203984: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-10-26 13:14:30.252274: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


269/269 [==============================] - 42s 150ms/step - loss: 0.4513 - accuracy: 0.8524 - val_loss: 0.1035 - val_accuracy: 0.9739
Epoch 2/3
269/269 [==============================] - 39s 145ms/step - loss: 0.1219 - accuracy: 0.9606 - val_loss: 0.0773 - val_accuracy: 0.9722
Epoch 3/3
91/91 [==============================] - 3s 35ms/step - loss: 0.0347 - accuracy: 0.9911
Mobilenetv2_Test accuracy: 99.11%


In [4]:
model.save("model100.h5")

/home/dharsan/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
